In [17]:
!pip install -q rouge-score


In [18]:
import json
from pathlib import Path
from rouge_score import rouge_scorer
import numpy as np

input_path = Path("val_candidates_constrained_reranked.json")
assert input_path.exists(), f"File not found: {input_path}"

with input_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

print("样本数量:", len(data))



样本数量: 1000


In [19]:
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

def rougeL_f1(pred: str, ref: str) -> float:
    """
    ROUGE-L F1 score for one sample.
    """
    score = scorer.score(ref, pred)
    return score["rougeL"].fmeasure



In [20]:
all_scores = []

for ex in data:
    reference = ex["reference"]

    # 已根据 NLI+SAS 排序，top-1 就是最佳摘要
    top1 = ex["candidates"][0]

    # 计算 ROUGE-L F1
    score = rougeL_f1(top1, reference)
    score_str = f"{score:.4f}"  # 保留四位小数

    # 将分数写在摘要前面，用 " || " 分隔
    new_summary = f"{score_str} || {top1}"

    # 写回 JSON：只保留这一条，并替换为带分数的版本
    ex["rougeL_f1"] = float(score)
    ex["candidates"] = [new_summary]

    all_scores.append(score)




In [21]:
output_path = Path("val_candidates_constrained_top1_with_rougeL.json")

with output_path.open("w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("已保存到:", output_path.resolve())


已保存到: /content/val_candidates_constrained_top1_with_rougeL.json


In [22]:
{
  "id": 12,
  "document": "...",
  "reference": "...",
  "candidates": [
    "0.3127 || The combined agency announced plans to ..."
  ],
  "rougeL_f1": 0.3127
}


{'id': 12,
 'document': '...',
 'reference': '...',
 'candidates': ['0.3127 || The combined agency announced plans to ...'],
 'rougeL_f1': 0.3127}